In [ ]:
import io
import os
from base64 import b64encode

import pandas as pd
import pyproj
from IPython.display import HTML, clear_output
from ipywidgets import Output, widgets

# Convert vertical datums

A simple application for converting Ground Control Point (GCP) elevations measured relative to [NN2000](https://no.wikipedia.org/wiki/NN2000) into elevations relative to the WGS84 ellipsoid.

## Input file format

The input file should be a **semicolon-separated text file without headers**, which is the format used by Pix4D for GCPs. As an example:

    GS18T_1;565025.763;6541468.499;1.798
    GS18T_2;565043.528;6541413.865;4.962
    GS18T_3;565074.924;6541368.255;0.506
    GS18T_4;565108.528;6541304.199;0.965
    GS18T_5;564998.028;6541340.293;0.489
    GS18T_6;565049.986;6541224.592;0.137
    GS18T_7;565067.664;6541156.521;0.625
    GS18T_8;565073.212;6541110.285;3.352
    GS18T_9;564925.444;6541548.647;0.326
    GS18T_10;564856.413;6541613.048;0.038
    GS18T_11;564831.512;6541642.163;0.186
    GS18T_12;565007.474;6541648.889;3.498
    GS18T_13;565021.855;6541612.919;0.188
    
Columns are as follows:

 * **Column 1:** A unique identified for each GCP
 * **Column 2:** `x` co-ordinates for each GCP expressed in ETRS89-based UTM co-ordinates
 * **Column 3:** `y` co-ordinates for each GCP expressed in ETRS89-based UTM co-ordinates
 * **Column 4:** GCP elevations in metres relative to NN2000
 
You must **remember to specify the correct UTM Zone** for the `x` and `y` co-ordinates when you upload the file.

## Output file format

The output file will be identical to the input file, except elevations in the fourth column will be expressed relative to the WGS84 ellipsoid (instead of NN2000).

In [ ]:
def trigger_download(fpath, filename, kind="text/json"):
    """Uses "data URIs". OK for small data files, but nothing too large.
    Adapted from here

        https://github.com/voila-dashboards/voila/issues/711#issuecomment-695872958
    """

    with open(fpath, "rb") as f:
        content_b64 = b64encode(f.read()).decode()
    data_url = f"data:{kind};charset=utf-8;base64,{content_b64}"
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """

    with download_output:
        clear_output()
        display(HTML(f"<script>{js_code}</script>"))


def download(e=None):
    """Adapted from here

    https://github.com/voila-dashboards/voila/issues/711#issuecomment-695872958
    """
    fname = upload_file.value[0]["name"]
    fname, ext = os.path.splitext(fname)
    fname = fname + "_converted" + ext
    fpath = os.path.join(r"/home/notebook", fname)
    trigger_download(fpath, fname, kind="text/plain")
    os.remove(fpath)
    upload_file.value = []
    upload_file._counter = 0
    reset(None)


def convert_coords(e=None):
    try:
        name = upload_file.value[0]["name"]
        content = upload_file.value[0]["content"]
        utm_zone = utm_dropdown.value
        # warning_text.value = ""
    except:
        msg = "Please upload a file."
        warning_text.value = f"<b><font color='red'>{msg}</b>"
        warning_text.layout.visibility = "visible"
        return None

    try:
        df = pd.read_csv(io.BytesIO(content), sep=";", names=["id", "x", "y", "z"])
        assert df.isna().sum().sum() == 0
        # warning_text.value = ""
    except:
        msg = "Error reading file. Please check the input specification above."
        warning_text.value = f"<b><font color='red'>{msg}</b>"
        warning_text.layout.visibility = "visible"
        return None

    # Convert elevations
    input_crs = 5940 + utm_zone
    src_crs = pyproj.crs.CRS.from_epsg(input_crs)
    wgs84_3d_crs = pyproj.crs.CRS.from_epsg(4979)
    transformer = pyproj.transformer.Transformer.from_crs(
        crs_from=src_crs, crs_to=wgs84_3d_crs, always_xy=True
    )
    x_wgs84, y_wgs84, z_wgs84 = transformer.transform(df.x, df.y, df.z)
    mean_offset = (z_wgs84 - df["z"]).mean()
    df["z"] = z_wgs84
    df["z"] = df["z"].round(3)

    # Save
    fname, ext = os.path.splitext(name)
    csv_path = os.path.join(r"/home/notebook", fname + "_converted" + ext)
    df.to_csv(csv_path, index=False, header=None, sep=";")

    # Enable download
    msg = f"Done.<br>Mean vertical offset: {mean_offset:.2f} m."
    success_text.value = f"<b><font color='green'>{msg}</b>"
    success_text.layout.visibility = "visible"
    download_button.layout.visibility = "visible"

    return df


def reset(e=None):
    warning_text.value = ""
    warning_text.layout.visibility = "hidden"

    success_text.value = ""
    success_text.layout.visibility = "hidden"

    download_button.layout.visibility = "hidden"

In [ ]:
# Initialise widgets
upload_file = widgets.FileUpload(
    description="Upload file", button_style="info", accept=".txt,.csv"
)
upload_file.observe(reset, names="value")

utm_dropdown = widgets.Dropdown(
    options=[31, 32, 33, 34, 35], value=33, description="UTM zone", disabled=False
)
utm_dropdown.observe(reset, names="value")

warning_text = widgets.HTML(value="")
warning_text.layout.visibility = "hidden"

convert_button = widgets.Button(description="Convert", button_style="info")
convert_button.on_click(convert_coords)

success_text = widgets.HTML(value="")
success_text.layout.visibility = "hidden"

download_button = widgets.Button(description="Download", button_style="info")
download_button.on_click(download)
download_button.layout.visibility = "hidden"

download_output = Output()
display(download_output)

# Widget layout
input_widgets = widgets.VBox(
    [
        widgets.HBox([upload_file, utm_dropdown]),
        widgets.HBox([warning_text]),
        widgets.HBox([convert_button]),
        widgets.HBox([success_text]),
        widgets.HBox([download_button]),
    ]
)

input_widgets